In [150]:
from datetime import datetime

import meteostat
import xarray as xr
from climepi import climdata
from climepi._xcdat import BoundsAccessor, center_times  # noqa

import opts

# Get options
options = opts.get_opts()
data_base_dir = options["data_base_dir"]
year_range = options["year_range"]
locations = [options["location_default"]] + options["locations_additional"]
no_locations = len(locations)

meteostat.Point.radius = 50000

In [163]:
# Get climate projection data (only to match longitudes and latitudes)
ds_clim = climdata.get_example_dataset("isimip_cities_daily", base_dir=data_base_dir)

In [194]:
ds_list = []

for location in locations:
    lat = ds_clim.sel(location=location)["lat"].values.item()
    lon = ds_clim.sel(location=location)["lon"].values.item()
    df_curr = meteostat.Daily(
        loc=meteostat.Point(lat=lat, lon=lon),
        start=datetime(2020, 1, 1),
        end=datetime(2020, 12, 31),
    ).fetch()
    ds_curr = (
        xr.Dataset.from_dataframe(df_curr[["tavg"]])
        .rename_vars(tavg="temperature")
        .assign_coords(lat=lat, lon=lon, location=location)
        .expand_dims("location", axis=1)
    )
    ds_list.append(ds_curr)

In [ ]:
ds = xr.concat(ds_list, dim="location")
ds = center_times(ds.bounds.add_time_bounds(method="freq", freq="day"))
ds["time"].encoding["calendar"] = "standard"
ds["time"].encoding["units"] = "hours since 2000-01-01 00:00:00"
ds

In [196]:
ds.to_netcdf(data_base_dir / "weather_2020.nc")